In [2]:
import os
import numpy as np
import datetime

def read_sdt_edit(s_path, s_file_name):
    s_file = os.path.join(s_path, s_file_name)
    s_out = s_file.replace('.SDT', '.his')
    s_out4 = s_file.replace('.SDT', '_225.csv')
    s_out5 = s_file.replace('.SDT', '_SPT.txt')

    if os.path.exists(s_file):
        nb = 0
        nspt = 0
        with open(s_file, 'rb') as fid, open(s_out, 'w') as fod, open(s_out4, 'w') as fod4, open(s_out5, 'w') as fid_spt:
            while True:
                hdr = np.fromfile(fid, dtype=np.uint8, count=5)
                nb += 5
                tms = np.fromfile(fid, dtype=np.uint8, count=6)
                if tms.size == 0:
                    break
                dt = get_timestamp(tms)
                bspt = np.fromfile(fid, dtype=np.uint8, count=512)
                bsys = np.fromfile(fid, dtype=np.uint8, count=32)
                chks = np.fromfile(fid, dtype=np.uint8, count=1)
                if get_checksum(np.concatenate((tms, bspt, bsys, chks))) == 0:
                    spt, sys = get_spectrum(bspt, bsys)
                    nspt += 1
                    prms, prms4 = get_wave_parameters(spt)

                    dts = dt.strftime("%Y-%m-%dT%H:%M:%S")
                    fod.write(f"{dts}, {', '.join(f'{p:.2f}' for p in prms[:11])}, "
                              f"{', '.join(f'{p:.3f}' for p in prms[11:15])}, {sys[4]:.2f}, {sys[5]:.2f}, {sys[6]}\n")
                    
                    fod4.write(f"{dts}\t{'\t'.join(str(p) for p in prms4)}\n")
                    fid_spt.write(f"Time Stamp= {dts}\t" + '\t'.join(f"{x:.3f}" for x in spt) + '\n')

                nb += 551

    return 0

def get_checksum(b):
    cs = b[0]
    for j in range(1, len(b)):
        cs ^= b[j]
    return cs

def get_timestamp(b):
    year = b[0] * 256 + b[1]
    month = b[2]
    day = b[3]
    hour = b[4]
    minute = b[5]
    return datetime.datetime(year, month, day, hour, minute)

def get_spectrum(bspt, bsys):
    GPS = (bsys[1] // 16) % 8
    Hm0 = ((bsys[2] * 256 + bsys[3]) % 4096) / 100
    Tz = 400 / ((bsys[4] * 256 + bsys[5]) % 256)
    Smax = np.exp(-0.005 * ((bsys[6] * 256 + bsys[7]) % 4096)) * 5000
    Tref = ((bsys[8] * 256 + bsys[9]) % 1024) / 20 - 5
    Tsea = ((bsys[10] * 256 + bsys[11]) % 1024) / 20 - 5
    Bat = bsys[12] % 8
    BLE = ((bsys[12] * 256 + bsys[13]) // 16) % 256
    Av = (bsys[14] * 256 + bsys[15]) % 4096
    Av = (2048 - Av if Av > 2048 else Av) / 800
    Ax = (bsys[16] * 256 + bsys[17]) % 4096
    Ax = (2048 - Ax if Ax > 2048 else Ax) / 800
    Ay = (bsys[18] * 256 + bsys[19]) % 4096
    Ay = (2048 - Ay if Ay > 2048 else Ay) / 800
    Lat = (((bsys[20] % 16) * 256 + bsys[21]) * 16 + (bsys[22] % 16)) * 256 + bsys[23]
    Lat = (Lat % (2**24)) / (2**23) * 90
    Lat = 90 - Lat if Lat > 90 else Lat
    Lon = (((bsys[24] % 16) * 256 + bsys[25]) * 16 + (bsys[26] % 16)) * 256 + bsys[27]
    Lon = (Lon % (2**24)) / (2**23) * 180
    Lon = 180 - Lon if Lon > 180 else Lon
    ori = ((bsys[28] * 256 + bsys[29]) % 4096) * 360 / 256
    incl = (bsys[31] + (bsys[30] % 16) / 16) * 360 / 256 / 2 - 90
    sys = [Hm0, Tz, Smax, Tref, Tsea, Bat, BLE, Av, Ax, Ay, GPS, Lat, Lon, ori, incl]

    spt = []
    for j in range(0, len(bspt), 8):
        jf = bspt[j] % 64
        frq = jf * 0.005 + 0.025 if jf < 16 else jf * 0.01 - 0.05
        sprlsb = bspt[j] // 64
        dir_angle = bspt[j + 1] * 360 / 256
        psd = np.exp(-0.005 * ((bspt[j + 2] * 256 + bspt[j + 3]) % 4096))
        n2lsb = (bspt[j + 2] // 16) % 4
        m2lsb = bspt[j + 2] // 64
        spr = (bspt[j + 4] + sprlsb / 4) * 360 / 256 / np.pi
        m2 = (bspt[j + 5] + m2lsb / 4) / 128 - 1
        n2 = (bspt[j + 6] + n2lsb / 4) / 128 - 1
        K = bspt[j + 7] * 0.01
        spt.append([frq, Smax * psd, get_wave_direction(dir_angle, spr, m2, n2), K, Lat, Lon])
    return np.array(spt), sys

def get_wave_direction(dir_angle, spr, m2, n2):
    sgmc = spr * np.pi / 180
    m1 = 1 - sgmc**2 / 2
    sgmca = np.sqrt((1 - m2) / 2)
    skw = -n2 / sgmca**3
    kurt = (6 - 8 * m1 + 2 * m2) / sgmc**4
    return [dir_angle, spr, skw, kurt]

def get_wave_parameters(spt):
    mom = np.zeros(8)
    mom2 = np.zeros(14)
    frq = spt[:, 0]
    psd = spt[:, 1]
    o = 3
    oo = 5
    for j in range(1, len(frq)):
        for n in range(-2, 5):
            yj1 = psd[j - 1] * frq[j - 1]**n
            yj = psd[j] * frq[j]**n
            mom[o + n] += 0.5 * (yj + yj1) * (frq[j] - frq[j - 1])
        for n in range(-4, 9):
            yj1 = psd[j - 1]**2 * frq[j - 1]**n
            yj = psd[j]**2 * frq[j]**n
            mom2[oo + n] += 0.5 * (yj + yj1) * (frq[j] - frq[j - 1])

    Hm0 = 4 * np.sqrt(mom[o])
    TI = np.sqrt(mom[o - 2] / mom[o])
    TE = mom[o - 1] / mom[o]
    T1 = mom[o] / mom[o + 1]
    Tz = np.sqrt(mom[o] / mom[o + 2])
    T3 = np.sqrt(mom[o + 1] / mom[o + 3])
    T4 = (mom[o + 1] / mom[o + 4]) ** 0.25
    Gamma = mom[o + 2] * mom[o] / mom[o + 1] ** 2
    sigH = np.sqrt(mom2[oo] - mom[o] ** 2)
    skew = (mom2[oo + 1] - 3 * mom[o] ** 2) / sigH ** 3
    kurt = (mom2[oo + 2] - 4 * mom2[oo + 1] * mom[o] + 6 * mom[o] ** 2 - 3 * mom[o] ** 4) / sigH ** 4

    return [Hm0, TI, TE, T1, Tz, T3, T4, Gamma, sigH, skew, kurt], [Hm0, TI, TE, T1]



In [4]:
!pwd
s_path= "/mnt/e/OneDrive/WORK/Projects/WRB_Dataprocessing"
s_file_name="Vizag_000001_S01-2012.SDT"
read_sdt_edit(s_path, s_file_name)

/mnt/e/OneDrive/WORK/Projects/WRB_Dataprocessing/test


ValueError: setting an array element with a sequence. The requested array has an inhomogeneous shape after 2 dimensions. The detected shape was (64, 6) + inhomogeneous part.